<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/detecting_diabetic_retinopathy_CNN_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

detecting diabetic retinopathy using a Convolutional Neural Network (CNN) with transfer learning from a pre-trained ResNet model. This code assumes you have a dataset of retinal fundus images labeled for different stages of diabetic retinopathy.

Step 1: Install Required Libraries

In [ ]:
!pip install tensorflow numpy matplotlib

Step 2: Python Code for Diabetic Retinopathy Detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load and preprocess data
# Assuming data is in './data/train' and './data/validation'
train_dir = './data/train'
validation_dir = './data/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Adding custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  # assuming 5 classes for retinopathy levels

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


Explanation:

Data Preparation: The data is loaded and augmented to increase the robustness of the model against various transformations in new unseen images.

Model Setup: ResNet50, a pre-trained CNN, is used as the base model with its convolutional base frozen to preserve learned features. Custom dense layers are added on top for classification specific to this task.

Training: The model is trained on the labeled dataset using the Adam optimizer and categorical crossentropy loss.

Evaluation: The training process is visualized by plotting the training and validation accuracy over epochs.

This approach leverages transfer learning to reduce training time and improve performance, utilizing ResNet50's robust feature extraction capabilities tailored to the specific task of diabetic retinopathy classification.





